to calculate the entropy of a node formula is E=-SUMMATIONP(X).LOG(

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter

In [5]:
class Node:
  def __init__(self, feature=None , threshold=None , left=None , right=None,*,value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value
  def is_leaf_node(self):
    return self.value is not None

In [8]:
import numpy as np
from collections import Counter

class DecisionTree:
  def __init__(self, min_samples_split=2, max_depth=100 , n_features=None ):
    self.min_samples_split=min_samples_split
    self.max_depth=max_depth
    self.n_features=n_features
    self.root=None

  def fit(self , x,y):
    self.n_features=x.shape[1] if not self.n_features else min(x.shape[1],self.n_features)
    self.root=self._grow_tree(x,y, depth=0)

  def _grow_tree(self,x,y, depth):

    n_samples,n_feats=x.shape
    n_labels = len(np.unique(y))
    #check the stopping criteria
    if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
      leaf_value=self._most_common_label(y)
      return Node(value=leaf_value)

    # Added check to prevent index error when n_features > n_feats
    if self.n_features > n_feats:
        self.n_features = n_feats
    feat_idx = np.random.choice(n_feats , self.n_features, replace = False)


    #find the best split
    best_tresh , best_feature_idx = self._best_split(x,y,feat_idx)

    #create child nodes
    left_idxs , right_idxs = self._split(x[:,best_feature_idx],best_tresh)
    left=self._grow_tree(x[left_idxs,:],y[left_idxs],depth+1)
    right = self._grow_tree(x[right_idxs,:],y[right_idxs],depth+1)
    return Node(best_feature_idx,best_tresh,left,right)

  def _best_split(self , x,y, feat_idxs):
    best_gain = -1
    split_idx , split_threshhold = None ,None
    for feat_idx in feat_idxs:
      x_column = x[:,feat_idx]
      thresholds = np.unique(x_column)
      for thr in thresholds :
        gain= self._information_gain(y, x_column, thr)

        if gain>best_gain:
          best_gain=gain
          split_idx=feat_idx
          split_threshhold=thr
    return split_threshhold , split_idx

  def _information_gain(self , y,x_column, threshold):
    #parent entropy
    parent_entropy = self._entropy(y)
    #create children
    left_idx, right_idx = self._split(x_column, threshold)
    if len(left_idx)==0 or len(right_idx)==0:
      return 0
    #calculate the weighted avg. entropy of children
    n=len(y)
    n_l, n_r = len(left_idx), len(right_idx)
    e_l, e_r = self._entropy(y[left_idx]), self._entropy(y[right_idx])
    child_entropy = (n_l/n)*e_l + (n_r/n)*e_r

    #calculate the IG
    information_gain = parent_entropy - child_entropy
    return information_gain



  def _split(self, x_column, split_thresh):
    left_idx = np.argwhere(x_column <= split_thresh).flatten()
    right_idx = np.argwhere(x_column > split_thresh).flatten()
    return left_idx, right_idx


  def _entropy(self,y):
    hist=np.bincount(y)
    ps=hist/len(y)
    return -np.sum([p*np.log(p) for p in ps if p>0])



  def _most_common_label(self,y):
    counter=Counter(y)
    value=counter.most_common(1)[0][0]
    return value


  def predict(self ,x):
    return np.array([self._traverse_tree(x_i, self.root) for x_i in x])

  def _traverse_tree(self,x,node):
    if node.is_leaf_node():
      return node.value
    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x,node.left)
    return self._traverse_tree(x,node.right)

In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter


data = datasets.load_breast_cancer()
x,y = data.data , data.target
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=1234)

clf = DecisionTree()
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
acc = np.sum(y_pred==y_test)/len(y_test)

def accuracy(y_test,y_pred):
  return np.sum(y_pred==y_test)/len(y_test)

acc = accuracy(y_test,y_pred)
print(acc)

0.9298245614035088
